<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

<img src="mioti.png" style="height: 100px">
<center style="color:#888">Data Science with Python</center>

# DSPy7 Challenge. Scrapy

Vamos a intentar extraer datos de una web de una librería, que no es más que una web preparada para testear scrapping contra ella. 

<img src="books_to_scrape.png" style="height: 500px">

Empezaremos concretamente en esta url: http://books.toscrape.com/catalogue/page-1.html

**Inicialización**

In [1]:
import logging
import numpy as np
import re
import pandas as pd
import json
import scrapy
from scrapy.crawler import CrawlerProcess
from scrapy.spiders import CrawlSpider, Rule
from scrapy.linkextractors import LinkExtractor
# Para ir viendo la salida de los comandos a medida que ocurre
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

**Esqueleto de los web scrapers**

In [2]:
class JsonWriterPipeline(object):
    @classmethod
    def from_crawler(cls, crawler):
        settings = crawler.settings
        file_name = settings.get("FILE_NAME")
        return cls(file_name)
    
    def __init__(self, file_name):
        self.file_name = file_name

    def open_spider(self, spider):
        print(self.file_name)
        self.file = open(self.file_name, 'w')

    def close_spider(self, spider):
        self.file.close()

    def process_item(self, item, spider):
        line = json.dumps(dict(item)) + "\n"
        self.file.write(line)
        return item

In [3]:
class SimpleBookSpider(scrapy.Spider):
    name = "simplebooks"
    start_urls = [
        'http://books.toscrape.com/catalogue/page-1.html',
    ]
    base_url = 'http://books.toscrape.com/'
    custom_settings = {
        'LOG_LEVEL': logging.WARNING,
        'ITEM_PIPELINES': {'__main__.JsonWriterPipeline': 1},     
        'FILE_NAME': "simplebook.jl",
        #'DEPTH_LIMIT': 2
    }

    def parse(self, response):
    
        current_page = int(re.findall(r'(\d+).html', response.url)[0])
        #print(current_page)
        results = response.xpath('//article[@class="product_pod"]')
        for t in results:
            yield {
                'title':t.xpath('.//h3/a/@title').extract_first(),
                'price':t.xpath('.//div/p[@class="price_color"]/text()').extract_first(),
                'rating':t.xpath('./p[contains(@class, "star-rating")]/@class').extract_first().replace('star-rating ', ''),
                'page': current_page
                 }
        if current_page < 3:
            next_page = response.xpath('//li[@class="next"]/a/@href').extract_first()
            if next_page is not None:
                yield response.follow(next_page, callback=self.parse)

In [4]:
#process = CrawlerProcess()

#process.crawl(SimpleBookSpider)
#process.crawl(RatingBookSpider)
#process.crawl(BookSpider)
#process.crawl(FinalBookSpider)
#process.start()

In [5]:
#dfjson = pd.read_json('simplebook.jl', lines=True)
#def change_to_number(x):
#    if x == 'One':
#        return 1
#    if x == 'Two':
#        return 2
#    if x == 'Three':
#        return 3
#    else:
#        return 4
#
#dfjson['rating'] = dfjson['rating'].apply(change_to_number)
#df_sorted = dfjson.sort_values(["rating"], ascending = (False))
#df_sorted

In [6]:
class FinalBookSpider(scrapy.Spider):
    name = "books"
    start_urls = [
        'http://books.toscrape.com/',
    ]
    custom_settings = {
        'LOG_LEVEL': logging.WARNING,
        'ITEM_PIPELINES': {'__main__.JsonWriterPipeline': 1},      
        'FILE_NAME': "finalbook.jl"
    }
    
    def parse(self, response):
            category_counter = 0
            for category in response.xpath('//div[@class="side_categories"]//li/a'):
                category_counter +=1
                if category_counter > 3:
                    break
                else:
                    category_url = category.xpath('@href').extract_first()
                    request = response.follow(url=category_url, callback=self.parse_category)
                    request.meta['category'] = category.xpath('text()').extract_first().strip()
                    yield request
    
    def parse_category(self,response):
        try:
            current_page = int(re.findall(r'(\d+).html', response.url)[0])
        except:
            current_page = 1
        for book in response.xpath('//article[@class="product_pod"]/h3/a'):
            book_url = book.xpath('@href').extract_first()
            request = response.follow(url=book_url, callback=self.parse_book)
            request.meta['page'] = current_page
            request.meta['category'] = response.meta.get('category')
            yield request
        next_page = response.xpath('//li[@class="next"]/a/@href').extract_first()
        if next_page is not None:
            request = response.follow(url=next_page, callback=self.parse_category)
            request.meta['category'] = response.meta.get('category')
            yield request
     
    def parse_book(self,response):
        
        yield {
            'category': response.meta.get("category"),
            'page': response.meta.get("page"),
            #'title': response.xpath('//article[@class="product_pod"]//h3/a/@title')
        }
            

In [7]:
process = CrawlerProcess()

#process.crawl(SimpleBookSpider)
#process.crawl(RatingBookSpider)
#process.crawl(BookSpider)
process.crawl(FinalBookSpider)
process.start()

2020-04-17 11:29:52 [scrapy.utils.log] INFO: Scrapy 2.0.1 started (bot: scrapybot)
2020-04-17 11:29:52 [scrapy.utils.log] INFO: Versions: lxml 4.4.1.0, libxml2 2.9.9, cssselect 1.1.0, parsel 1.5.2, w3lib 1.20.0, Twisted 20.3.0, Python 3.7.4 (default, Aug 13 2019, 15:17:50) - [Clang 4.0.1 (tags/RELEASE_401/final)], pyOpenSSL 19.0.0 (OpenSSL 1.1.1d  10 Sep 2019), cryptography 2.7, Platform Darwin-17.4.0-x86_64-i386-64bit
2020-04-17 11:29:52 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2020-04-17 11:29:52 [scrapy.crawler] INFO: Overridden settings:
{'LOG_LEVEL': 30}


finalbook.jl


In [8]:
dfjson = pd.read_json('finalbook.jl')
dfjosn

ValueError: Trailing data

**Ejercicio 1:** Obten en un dataframe una lista de los libros en esta página (no en toda la librería), con título y precio.

**Ejercicio 2:** Ahora captura también el rating de los libros. ¡Ordénalos por rating!

**Ejercicio 3:** Vayamos un poco más lejos y expandamos el mismo Spider para que siga los enlaces de siguiente. Obten una lista de las 3 primeras páginas con libros y construye un dataframe en el que además, por cada libro, aparezca el número de página en que aparece. ¡¡¡Prohibido usar "start_urls"!!!

**Ejercicio 4:** El spider definitivo. Construye una clase que vaya categoria por categoría de libros (para hacer la ejecución más agil, basta con que solo lo haga con las 4 primeras). Por cada categoría, que entre en cada libro y obtenga:
* Título
* Precio
* Libros en stock
* Rating
* UPC
* Imagen

Ojo, porque una categoría puede tener más de una página y quiero los libros de todas 😠.